In [100]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/jf/8m56dyc948d8v7l351v4ggmc0000gn/T/ipykernel_65935/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [101]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [102]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [103]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


## Data Preprocessing

In [104]:
#import nltk
#nltk.download('stopwords')

In [105]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [106]:
import re
def clean_html(html):
    """Copied from NLTK package.
    Remove HTML markup from the given string.

    :param html: the HTML string to be cleaned
    :type html: str
    :rtype: str"""
    # First we remove inline JavaScript/CSS:
    cleaned = html.strip()
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", " ", cleaned)
    
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", " ", cleaned)
    
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)   
    

    return cleaned.strip()

In [107]:
for i in range(len(data)):
    data.iloc[i, 0]  = clean_html(data.iloc[i,0])

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [108]:

def remove_special_characters(text, keep_spaces=False):
    """
    Removes special characters from a given string.
    
    Args:
    text (str): The input string to remove special characters from.
    keep_spaces (bool, optional): If True, it keeps space characters in the output string.
                                  If False, it removes all spaces too. Default is False.
    
    Returns:
    str: The input string with special characters removed.
    """
    special_chars = set(string.punctuation)
    
    if keep_spaces:
        special_chars.remove(' ')
    
    translation_table = str.maketrans('', '', ''.join(special_chars))
    
    cleaned_text = text.translate(translation_table)
    
    return cleaned_text

In [109]:
for i in range(len(data)):
    data.iloc[i, 0]  = remove_special_characters(data.iloc[i,0])


In [110]:

def remove_numbers(text):
    # Use a regular expression to replace all digits with an empty string
    cleaned_text = re.sub(r'\d+', '', text)
    
    return cleaned_text


In [111]:
for i in range(len(data)):
    data.iloc[i, 0]  = remove_numbers(data.iloc[i,0])

In [112]:

def remove_single_chars(text):
    """
    Removes all single characters from a given string.
    
    Args:
    text (str): The input string to remove single characters from.
    
    Returns:
    str: The input string with single characters removed.
    """
    # Split the string into a list of words
    words = text.split()
    
    # Filter out single-character words
    cleaned_words = [word for word in words if len(word) > 1]
    
    # Join the filtered words back into a string
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

In [113]:
for i in range(len(data)):
    data.iloc[i, 0]  = remove_single_chars(data.iloc[i,0])

In [114]:

def remove_single_chars_start(text):
    """
    Removes single characters from the start of a given string.
    
    Args:
    text (str): The input string to remove single characters from the start.
    
    Returns:
    str: The input string with single characters removed from the start.
    """
    # Strip single characters from the start of the string
    cleaned_text = text.lstrip(''.join(set(text) - set(' ')))
    
    return cleaned_text


In [115]:
for i in range(len(data)):
    data.iloc[i, 0]  = remove_single_chars_start(data.iloc[i,0])

In [116]:

def clean_text(text):
    """
    Substitute multiple spaces with single space, remove prefixed 'b',
    and convert to lowercase.

    Args:
    text (str): The input string to clean.

    Returns:
    str: The cleaned string.
    """
    # Substitute multiple spaces with single space
    cleaned_text = re.sub(r'\s+', ' ', text)

    # Remove prefixed 'b'
    cleaned_text = cleaned_text.lstrip('b')

    # Convert to lowercase
    cleaned_text = cleaned_text.lower()

    return cleaned_text



In [117]:
for i in range(len(data)):
    data.iloc[i, 0]  = clean_text(data.iloc[i,0])

In [118]:
data.head()

,text,label
0,sir strictly private business proposal am mik...,1
1,do,0
2,has emailed dozens of memos about haiti to me...,0
3,sirfmadamc know that this proposal might be s...,1
4,,0


## Now let's work on removing stopwords
Remove the stopwords.

In [95]:
# Load the stop words set
stop_words = stopwords.words("english")


In [120]:
#filtered_words = [word for word in data if word not in stopwords.words('english')]

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))


In [121]:
data.head()

,text,label
0,sir strictly private business proposal mike ch...,1
1,,0
2,emailed dozens memos haiti weekend please prin...,0
3,sirfmadamc know proposal might surprise emerge...,1
4,,0


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [131]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [134]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)


In [136]:
data['text'] = data["text"].apply(lemmatize_text)

,text,label
0,sir strictly private business proposal mike ch...,1
1,,0
2,emailed dozen memo haiti weekend please print ...,0
3,sirfmadamc know proposal might surprise emerge...,1
4,,0


In [139]:
data.head(20)

,text,label
0,sir strictly private business proposal mike ch...,1
1,,0
2,emailed dozen memo haiti weekend please print ...,0
3,sirfmadamc know proposal might surprise emerge...,1
4,,0
5,bottom line need special security code get lin...,0
6,siri engr ugo nzego engineering store departme...,1
7,huma saturday november pmhfw quint fmsim going...,0
8,oct th george marshall event department tentat...,0
9,account owner colleague set aside defray incid...,1


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [138]:
data.groupby(by="label").sum()

,text
label,
0,emailed dozen memo haiti weekend please print ...
1,sir strictly private business proposal mike ch...


## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code